In [5]:
from subprocess import Popen, PIPE, STDOUT
from random import randint
import binascii
import os
from queue import Queue, Empty
from threading  import Thread
from time import sleep
import random
import nltk
nltk.download('averaged_perceptron_tagger')
from itertools import permutations, combinations
import spacy
nlp = spacy.load('en')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sparr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [63]:
basic_actions = ['up',
           'down',
           'look',
           'climb',
           'go north',
           'go south',
           'go west',
           'go east',
           'go northeast',
           'go northwest',
           'go southeast',
           'go southwest',
           'take all',
           'inventory']

action_space = set()

for action in basic_actions:
    action_space.add(action)

command1_actions = ['take x',
                   'open x',
                   'read x',
                   'drop x',
                   'turn on x',
                   'turn off x',
                   'move x',
                   'eat x',
                   'close x',
                   'drink x',
                   'smell x',
                   'listen x',
                   'climb x']
                  

command2_actions = ['throw x at y',
                    'put x in y',
                    'turn x with y',
                    'attack x with y',
                    'tie x with y',
                    'break x with y',
                    'cut x with y']
                    

filtered_tokens = ['Score', 'Moves']

In [7]:
def enqueue_output(out, queue):
    for line in iter(out.readline, b''):
        queue.put(line)
    out.close()

In [8]:
def start_game():
    score = 0
    moves = 0
    p = Popen(['dfrotz.exe','zork1.z5'], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    q = Queue()
    t = Thread(target=enqueue_output, args=(p.stdout, q))
    t.daemon = True # thread dies with the program
    t.start()
    sleep(0.1)
    return(p, q, t, score, moves)

In [9]:
# read line without blocking
def readLine(q):
    cont = True
    narrative = ""
    while cont:
        try:  line = q.get_nowait() # or q.get(timeout=.1)
        except Empty:
            cont = False
        else: 
            narrative = narrative + line.decode("utf-8").replace('\n', " ").replace('\r', " ")
    if ('840726' in narrative): ## Opening narrative
        narrative = narrative[narrative.index('840726') + len('840726'):]
    sleep(0.1)
    return(narrative)

In [10]:
def grab_score_moves(narrative):
    try:
        score = int(narrative[narrative.index('Score: ') + len('Score: '):][0:3].strip())
        moves = int(narrative[narrative.index('Moves: ') + len('Moves: '):][0:3].strip())
    except:  ## not valid move
        score = 0
        moves = 0
    return(score, moves)

In [11]:
def grab_story(narrative):
    if ('Moves:' in narrative):
        story = narrative[narrative.index('Moves:') + len('Moves:'):].strip()
    else:
        story = narrative.strip()
    return story

In [29]:
def preprocess(text): 
    text = nltk.word_tokenize(text)
    processed = nltk.pos_tag(text)
    return processed

In [39]:
def detect_nouns(narrative):  #grabs nouns and adjective noun pairs from narrative
    
    processed = preprocess(narrative)
    pattern = 'NP: {<JJ>*<NN>}'
    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(processed)
    noun_list =  [noun for noun in nlp(narrative).noun_chunks]
    return(noun_list)
    
    # function to test if something is a noun
    #is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    #tokenized = nltk.word_tokenize(narrative)
    #nouns = [word.lower() for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    #return(nouns)

In [13]:
def generate_action_tuples(nouns):
    possible_actions = []
    for i in nouns:
        for action1 in command1_actions:   ## first loop replaces 'x' in each action in command1_actions
            action_to_add = action1.replace('x', i)
            possible_actions.append(action_to_add)
        noun_permutations = list(permutations(nouns, 2))    ## second loop replaces 'x' and 'y' in each action in command2_actions
        for action2 in command2_actions:
            for perm in noun_permutations:
                if (perm[0] == perm[1]):  ## ignore same noun acting on itself
                    pass
                else:
                    action_to_add = action2.replace('x', perm[0])
                    action_to_add = action_to_add.replace('y', perm[1])
                    possible_actions.append(action_to_add)
    return possible_actions

In [14]:
def add_to_action_space(action_space, actions):
    for action in actions:
        action_space.add(action)
    return action_space

In [15]:
def perform_action(command, p):
    p.stdin.write(bytes(command+ "\n", 'ascii'))
    p.stdin.flush()
    sleep(0.1) ## wait for action to register

In [16]:
def kill_game(p):
    p.terminate()
    p.kill()

In [17]:
def choose_random_action(p, q, action_space):
    action = random.sample(action_space, 1)
    return action[0]

In [19]:
p, q, t, score, moves = start_game()

In [20]:
nar = readLine(q)

In [62]:
detect_nouns(nar)

[What, you, the self]

In [60]:
perform_action('commit suicide', p)

In [61]:
nar = readLine(q)
nar

'>What do you want to kill the self with?    '

In [ ]:
perform

In [154]:
p, q, t, score, moves = start_game()
actions = []
narratives = []
scores = []
moves_count = 0
messages = []
action_narrative = []
for i in range(0,500):
    narrative = readLine(q)
    nouns = detect_nouns(message)
    current_action_space = generate_action_tuples(nouns)
    action_space = add_to_action_space(action_space, current_action_space)
    action = choose_random_action(p,q, action_space)
    perform_action(action, p)
    
    actions.append(action)
    narratives.append(narrative)
    action_narrative.append([narrative, action])
    score, moves = grab_score_moves(narrative)
    scores.append(score)
    moves_count += 1
    print("Round: " + str(moves_count) + "  -  Score: " + str(score))

Round: 1  -  Score: 0
Round: 2  -  Score: 0
Round: 3  -  Score: 0
Round: 4  -  Score: 0
Round: 5  -  Score: 0
Round: 6  -  Score: 0
Round: 7  -  Score: 0
Round: 8  -  Score: 0
Round: 9  -  Score: 0
Round: 10  -  Score: 0
Round: 11  -  Score: 0
Round: 12  -  Score: 0
Round: 13  -  Score: 0
Round: 14  -  Score: 0
Round: 15  -  Score: 0
Round: 16  -  Score: 0
Round: 17  -  Score: 0
Round: 18  -  Score: 0
Round: 19  -  Score: 0
Round: 20  -  Score: 0
Round: 21  -  Score: 0
Round: 22  -  Score: 0
Round: 23  -  Score: 0
Round: 24  -  Score: 0
Round: 25  -  Score: 0
Round: 26  -  Score: 0
Round: 27  -  Score: 0
Round: 28  -  Score: 0
Round: 29  -  Score: 0
Round: 30  -  Score: 0
Round: 31  -  Score: 0
Round: 32  -  Score: 0
Round: 33  -  Score: 0
Round: 34  -  Score: 0
Round: 35  -  Score: 0
Round: 36  -  Score: 0
Round: 37  -  Score: 0
Round: 38  -  Score: 0
Round: 39  -  Score: 0
Round: 40  -  Score: 0
Round: 41  -  Score: 0
Round: 42  -  Score: 0
Round: 43  -  Score: 0
Round: 44  -  Score:

Round: 347  -  Score: 0
Round: 348  -  Score: 0
Round: 349  -  Score: 0
Round: 350  -  Score: 0
Round: 351  -  Score: 0
Round: 352  -  Score: 0
Round: 353  -  Score: 0
Round: 354  -  Score: 0
Round: 355  -  Score: 0
Round: 356  -  Score: 0
Round: 357  -  Score: 0
Round: 358  -  Score: 0
Round: 359  -  Score: 0
Round: 360  -  Score: 0
Round: 361  -  Score: 0
Round: 362  -  Score: 0
Round: 363  -  Score: 0
Round: 364  -  Score: 0
Round: 365  -  Score: 0
Round: 366  -  Score: 0
Round: 367  -  Score: 0
Round: 368  -  Score: 0
Round: 369  -  Score: 0
Round: 370  -  Score: 0
Round: 371  -  Score: 0
Round: 372  -  Score: 0
Round: 373  -  Score: 0
Round: 374  -  Score: 0
Round: 375  -  Score: 0
Round: 376  -  Score: 0
Round: 377  -  Score: 0
Round: 378  -  Score: 0
Round: 379  -  Score: 0
Round: 380  -  Score: 0
Round: 381  -  Score: 0
Round: 382  -  Score: 0
Round: 383  -  Score: 0
Round: 384  -  Score: 0
Round: 385  -  Score: 0
Round: 386  -  Score: 0
Round: 387  -  Score: 0
Round: 388  -  S

In [155]:
action_narrative

[['    West of House  You are standing in an open field west of a white house, with a boarded  front door.  There is a small mailbox here.    ',
  'tie mailbox with house'],
 [">That sentence isn't one I recognize.    ", 'throw door at west'],
 [">You don't have the door.    ", 'cut west with mailbox'],
 ["> West of House                               Score: 0        Moves: 1    You can't see any west here!    ",
  'tie house with field'],
 ['>I don\'t know the word "field".    ', 'tie west with mailbox'],
 [">That sentence isn't one I recognize.    ", 'cut west with house'],
 ["> West of House                               Score: 0        Moves: 2    You can't see any west here!    ",
  'down'],
 ["> West of House                               Score: 0        Moves: 3    You can't go that way.    ",
  'close field'],
 ['>I don\'t know the word "field".    ', 'attack west with door'],
 [">You don't have the door.    ", 'drop field'],
 ['>I don\'t know the word "field".    ', 'turn on d

In [153]:
kill_game(p)


In [138]:
p, q, t, score, moves = start_game()

In [150]:
narrative = readLine(q)

In [151]:
grab_story(narrative)

''

In [46]:
nouns = detect_nouns(narrative)
nouns

['west', 'house', 'field', 'west', 'house', 'front', 'door', 'mailbox']

In [49]:
action_space = generate_action_tuples(nouns)


In [148]:
make_random_play(p, q, action_space)

eat west


('eat west',
 "> West of House                               Score: 0        Moves: 1    You can't see any west here!    ")

In [143]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 50

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                    .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

ModuleNotFoundError: No module named 'gym'